# Problem logarytmu dyskretnego

Niech $(G,\circ)$ będzie grupą z działaniem $\circ$ i elementem neutralnym $1_G$. Wtedy dla dowolnego elementu $a\in G$ i $k\in\mathbb{Z}$ definiujemy *potęgę* $$a^k =\left\{\begin{array}{cc}
\underbrace{a\circ a\circ \ldots \circ a}_{k}&\text{ dla }k>0\\
1_G&\text{ dla }k=0\\
\underbrace{a^{-1}\circ a^{-1}\circ \ldots \circ a^{-1}}_{k}&\text{ dla }k<0
\end{array}\right.$$
gdzie $a^{-1}$ jest elementem odwrotnym do $a$.

Dla $a,b\in G$, $b\neq 1_G$, *logarytmem dyskretnym* $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a$.

## Logarytm dyskretny w $\mathbb{Z}_n$

W przypadku pierścienia $\mathbb{Z}_n$ logarytmem dyskretnym $\log_b a$ jest każda liczba $k\in\mathbb{Z}$ taka, że $b^k=a\mod n$, o ile w ogóle istnieje.

Specyficzną sytuacją w teorii liczb jest gdy $n=p$ jest liczbą pierwszą a $q$ jest pierwiastkiem pierwotnym $\mod p$. Wtedy:
- potęgi $q^k\mod p$ generują cały zbiór $[1,p-1]$, tzn. $q$ jest generatorem grupy multiplikatywneh rzędu $p-1$
- logarytm dyskretny $\log_q a$ istnieje dla każdego niezerowego elementu $a\in \mathbb{Z}_p$

## Algorytm baby-step giant-step

Jeden z najprostszych (poza metodą naiwną) algorytmów poszukiwania logarytmu dyskretnego w grupach cyklicznych.

Niech $p$ będzie liczbą pierwszą oraz niech $q$ będzie pierwiastkiem pierwotnym modulo $p$. Dla niezerowego $a\in\mathbb{Z}_p$ szukamy liczby $k\in\mathbb{Z}$ takiej, że $q^k=a\mod p$

### Krok 1.
- $m=\lceil\sqrt{p-1}\rceil$
- tworzymy pomocniczą tablicę potęg: dla wszystkich $i\in [0,m)$ obliczamy parę $(i,q^i)$
- obliczamy $r=(q^{-1})^m$
### Krok 2.
- $b=a$
- dla wszystkich $j\in [0,m)$:
    - sprawdzamy, czy para $(i,b)$ jest elementem tablicy potęg dla pewnego $i$
    - jeżeli tak, to $k=jm+i$ i kończymy algorytm
    - jeżeli nie, to $b=br$ i kontynuujemy pętlę


## Zadanie 1.

Zaimplementować algorytm baby-step giant-step. Przetestować dla podanych danych testujących.

```Dane testujące:
p = 7
q = 3
a = 4

m = 3
tablica_testowa = [1,3,2]
r = 6
k = 4 (j = 1, i = 1)
```

```
p = 29
q = 8
a = 10

m = 6
tablica_testowa = [1,8,6,19,7,27]
r = 9
k = 17 (j = 2, i = 5)
```

```
p = 113
q = 76
a = 84

m = 11
tablica_testowa = [1,76,13,84,56,75,50,71,85,19,88]
r = 70
k = 3 (j = 0, i = 3)
```

In [8]:
from math import sqrt, ceil

def baby_step(p, q, debug=True):
    m = ceil(sqrt(p - 1))
    if debug:
        print(f"m = {m}")
    test_table = {}
    for i in range(m):
        test_table[i] = pow(q, i, p)
    if debug:
        print(f"tablica_testowa = {test_table}")
    r = pow(q, -m, p)
    
    if debug:
        print(f"r = {r}")

    return m, test_table, r

def giant_step(p, q, a, debug=True):
    m, test_table, r = baby_step(p, q, debug)
    b = a % p
    for j in range(m):
        for i in test_table:
            if b == test_table[i]:
                k = (j * m + i) % p
                if debug:
                    print(f"k = {k} (j = {j}, i = {i})")
                return k
        b = (b * r) % p
    return None

In [9]:
giant_step(7,3,4, False) == 4
giant_step(29,8,10, False) == 17
giant_step(113,76,84, False) == 3

True

# Wymiana klucza typu Diffie-Hellman

Alice i Bob uzgadniają klucz publiczny będący liczbą pierwszą $p$ oraz $q$ - pierwiastkiem pierwotnym mod $p$.
- sekret Alice: liczba całkowita $n\in \mathbb{Z}_p\setminus\{0\}$
- sekret Boba: liczba całkowita $m\in \mathbb{Z}_p\setminus\{0\}$
- Alice generuje $x=q^n\mod p$ i wysyła do Boba
- Bob generuje $y=q^m\mod p$ i wysyła Alice
- Alice oblicza klucz $k=y^n\mod p$
- Bob oblicza klucz $k=x^m\mod p$


## Zadanie 2.

Zaimplementuj powyższy algorytm wymiany klucza. Dobierz parametry $p$ i $q$ tak, żeby znając $x$, $y$, $p$ i $q$ nie dało się odtworzyć sekretów algorytmem z zadania 1.

In [10]:
from random import randint

def Diffie_Hellman(p,q):
    n = randint(1,p-1)
    m = randint(1,p-1)
    x = pow(q,n,p)
    y = pow(q,m,p)
    k_ALICE = pow(y,n,p)
    k_BOB = pow(x,m,p)

    print(f"Alice generated {n}")
    print(f"BOB generated {m}")
    print(f"Alice sent {x}")
    print(f"BOB sent {y}")
    print(f"Alice calculated k = {k_ALICE}")
    print(f"BOB calculated k = {k_BOB}")

    return x, y


def is_primitive_root(g, p):
    if p <= 2:
        return False

    phi = p - 1

    factors = []
    n = phi
    for i in range(2, int(n**0.5) + 1):
        if n % i == 0:
            factors.append(i)
            while n % i == 0:
                n //= i
    if n > 1:
        factors.append(n)

    for factor in factors:
        if pow(g, phi // factor, p) == 1:
            return False
    return True


def find_primitive_root(p):
    for g in range(2, p):
        if is_primitive_root(g, p):
            return g
    return None

In [11]:
p = 2137
primitive_root = find_primitive_root(p)
print(f"Primitive_root({p}): {primitive_root}")

Primitive_root(2137): 10


In [12]:
p = 2**41 - 1
primitive_root = find_primitive_root(p)
print(f"Primitive_root({p}): {primitive_root}")

Primitive_root(2199023255551): 3


In [13]:
print("Test case 1:")
p = 2137
q = find_primitive_root(p)

x,y = Diffie_Hellman(p,q)

print()
n = giant_step(p,q,x, debug=False)
m = giant_step(p,q,y, debug=False)


print(f"Found Alice secret {n}")
print(f"Found BOB secret {m}\n\n")

print("Test case 2:")
p = 12345
q = find_primitive_root(p)

x,y = Diffie_Hellman(p,q)

print()
n = giant_step(p,q,x, debug=False)
m = giant_step(p,q,y, debug=False)


print(f"Found Alice secret {n}")
print(f"Found BOB secret {m}\n\n")

print("Test case 3:")
p = 23
q = find_primitive_root(p)

x,y = Diffie_Hellman(p,q)

print()
n = giant_step(p,q,x, debug=False)
m = giant_step(p,q,y, debug=False)

print(f"Found Alice secret {n}")
print(f"Found BOB secret {m}\n\n")

print("Test case 4:")
p = 17
q = find_primitive_root(p)

x,y = Diffie_Hellman(p,q)

print()
n = giant_step(p,q,x, debug=False)
m = giant_step(p,q,y, debug=False)

print(f"Found Alice secret {n}")
print(f"Found BOB secret {m}\n\n")

print("Test case 5:")
p = 2**32 -1
q = find_primitive_root(p)

x,y = Diffie_Hellman(p,q)

print()
n = giant_step(p,q,x, debug=False)
m = giant_step(p,q,y, debug=False)

print(f"Found Alice secret {n}")
print(f"Found BOB secret {m}\n\n")

Test case 1:
Alice generated 88
BOB generated 875
Alice sent 16
BOB sent 1541
Alice calculated k = 884
BOB calculated k = 884

Found Alice secret 88
Found BOB secret 875


Test case 2:
Alice generated 11551
BOB generated 5219
Alice sent 6773
BOB sent 11588
Alice calculated k = 4517
BOB calculated k = 4517

Found Alice secret 43
Found BOB secret 287


Test case 3:
Alice generated 7
BOB generated 21
Alice sent 17
BOB sent 14
Alice calculated k = 19
BOB calculated k = 19

Found Alice secret 7
Found BOB secret 21


Test case 4:
Alice generated 13
BOB generated 4
Alice sent 12
BOB sent 13
Alice calculated k = 13
BOB calculated k = 13

Found Alice secret 13
Found BOB secret 4


Test case 5:
Alice generated 3383908551
BOB generated 368171029
Alice sent 128
BOB sent 2097152
Alice calculated k = 524288
BOB calculated k = 524288

Found Alice secret 7
Found BOB secret 21


